In [2]:
# Install ultralytics if not installed
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.2/915.2 kB 27.6 MB/s eta 0:00:00


In [6]:
import time
import os
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image

# Kaggle dataset directory (update with your dataset path)
dataset_path = "/kaggle/input/wadaba-models"

# Get all model files
yolo_models = [os.path.join(dataset_path, model) for model in os.listdir(dataset_path) if model.endswith(".pt")]
keras_models = [os.path.join(dataset_path, model) for model in os.listdir(dataset_path) if model.endswith(".keras")]

# Load test image
image_path = os.path.join(dataset_path, "test_image.jpg")
image = Image.open(image_path)
image_resized = image.resize((224, 224))  # Resize if needed for Keras models
image_array = np.array(image_resized) / 255.0  # Normalize
image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

# Measure YOLO inference speed
print("=== YOLO Model Inference Speed ===")
for model_path in yolo_models:
    model = YOLO(model_path)  # Load YOLO model

    start_time = time.time()
    results = model(image_path)  # Run inference
    end_time = time.time()

    inference_time = end_time - start_time
    fps = 1 / inference_time if inference_time > 0 else 0

    print(f"Model: {os.path.basename(model_path)}")
    print(f"Inference Time: {inference_time:.4f} seconds")
    print(f"FPS: {fps:.2f}\n")

# Measure Keras inference speed
print("=== Keras Model Inference Speed ===")
for model_path in keras_models:
    # Print model input shape
    model = load_model(keras_models[0])  
    print("Expected input shape:", model.input_shape)
    
    # Adjust image shape dynamically
    image_resized = image.resize((model.input_shape[1], model.input_shape[2]))  # Resize based on model
    image_array = np.array(image_resized) / 255.0  
    
    # If model expects 1D input, flatten the image
    if len(model.input_shape) == 2:
        image_array = image_array.reshape(1, -1)  # Flatten to match input shape
    else:
        image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    
    # Run inference
    start_time = time.time()
    predictions = model.predict(image_array)
    end_time = time.time()
    
    # Measure FPS
    inference_time = end_time - start_time
    fps = 1 / inference_time if inference_time > 0 else 0
    
    print(f"Inference Time: {inference_time:.4f} seconds")
    print(f"FPS: {fps:.2f}")



=== YOLO Model Inference Speed ===

image 1/1 /kaggle/input/wadaba-models/test_image.jpg: 448x640 1 PP, 1 PS, 1 Other, 628.5ms
Speed: 3.4ms preprocess, 628.5ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
Model: yolo10m_best.pt
Inference Time: 1.1151 seconds
FPS: 0.90


image 1/1 /kaggle/input/wadaba-models/test_image.jpg: 448x640 1 Other, 138.8ms
Speed: 3.0ms preprocess, 138.8ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)
Model: yolo10n_best.pt
Inference Time: 0.3027 seconds
FPS: 3.30


image 1/1 /kaggle/input/wadaba-models/test_image.jpg: 448x640 1 Other, 664.2ms
Speed: 2.9ms preprocess, 664.2ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)
Model: yolo11m_best.pt
Inference Time: 1.2082 seconds
FPS: 0.83


image 1/1 /kaggle/input/wadaba-models/test_image.jpg: 448x640 1 Other, 112.1ms
Speed: 2.8ms preprocess, 112.1ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)
Model: yolo11n_best.pt
Inference Time: 0.2608 